In [24]:
# set app as default directory to address imports
import os
import sys
sys.path.append(os.path.join(os.getcwd(), './../../'))

#activate autoreload to easier test classes
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from app.services.sparql_graph import SPARQLGraph
from app.config.enums import Environment
graph = SPARQLGraph(Environment.PROD, False)

Metadata loaded successfully from JSON files.
Initializing SPARQLGraph
Graph loaded with 2056777 triples after 0:02:14.016682


In [26]:
from rdflib import URIRef
import rdflib
WD = rdflib.Namespace('http://www.wikidata.org/entity/')
WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
DDIS = rdflib.Namespace('http://ddis.ch/atai/')
RDFS = rdflib.namespace.RDFS
SCHEMA = rdflib.Namespace('http://schema.org/')

In [29]:
graph.get_id_for_movie('The Matrix')

'http://www.wikidata.org/entity/Q83495'

In [11]:
graph.get_rel_for_lbl('occupation')

'http://www.wikidata.org/prop/direct/P106'

In [9]:
# let's see what our graph thinks the occupation of Jean Van Hamme is
professions = set(graph.graph.query('''
    prefix wdt: <http://www.wikidata.org/prop/direct/>
    prefix wd: <http://www.wikidata.org/entity/>
    
    SELECT ?obj ?lbl WHERE {
        ?ent rdfs:label "Jean Van Hamme"@en .
        ?ent wdt:P106 ?obj .
        ?obj rdfs:label ?lbl .
    }
    '''))
{ent[len(WD):]: str(lbl) for ent, lbl in professions}

{'Q329737': 'butcher'}

In [30]:
# Step 1: Retrieve the full URIs for entity and property using your functions
entity_id = graph.get_id_for_movie('The Matrix')   # Expected: "http://www.wikidata.org/entity/Q428160"
property_id = graph.get_rel_for_lbl('director')     # Expected: "http://www.wikidata.org/prop/direct/P106"

print(entity_id, property_id)
# Step 2: Query using the full URIs directly in the SPARQL query
occupation_query = set(graph.graph.query(f'''
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    
    SELECT ?obj ?lbl WHERE {{
        <{entity_id}> <{property_id}> ?obj .
        ?obj rdfs:label ?lbl .
        FILTER(LANG(?lbl) = "en")
    }}
    '''))

# Step 3: Format the results as a dictionary with entity IDs and labels
occupations = {obj.split('/')[-1]: lbl for obj, lbl in occupation_query}

# Display the occupations
occupations

http://www.wikidata.org/entity/Q83495 http://www.wikidata.org/prop/direct/P57


{'Q195719': rdflib.term.Literal('The Wachowskis', lang='en')}